In [ ]:
# ==== Part 2a: Normal Form + Computation (Colab-ready) ====
# Public Goods Game (2 players) with E=100, multiplier m=1.5, n=2
# Discretize contributions, build payoff matrices, compute Nash equilibria with NashPy

!pip -q install nashpy==0.0.38 pandas numpy

import numpy as np
import pandas as pd
import nashpy as nash

# -----------------------------
# 1) Parameters & grid
# -----------------------------
E = 100          # endowment per player
m = 1.5          # public good multiplier
n = 2            # group size
MPCR = m / n     # marginal per-capita return; here 0.75 < 1

# Coarse grid first; you can try finer grids later for robustness
grid = np.array([0, 25, 50, 75, 100], dtype=float)

print(f"Parameters: E={E}, m={m}, n={n}, MPCR={MPCR:.2f}")
print(f"Action grid: {grid.tolist()}")

# -----------------------------
# 2) Payoff function & matrices
# -----------------------------
def payoff_i(gi, gj, E=E, m=m, n=n):
    """
    Player i's payoff when i contributes gi and j contributes gj:
    pi = E - gi + (m/n)*(gi + gj)
    """
    return E - gi + (m/n)*(gi + gj)

# Build payoff matrices A (row player), B (column player)
A = np.zeros((len(grid), len(grid)))
B = np.zeros_like(A)

for i, g1 in enumerate(grid):
    for j, g2 in enumerate(grid):
        A[i, j] = payoff_i(g1, g2, E, m, n)            # row player's payoff
        B[i, j] = payoff_i(g2, g1, E, m, n)            # column player's payoff (symmetric)

# -----------------------------
# 3) Display payoff matrices with labels
# -----------------------------
dfA = pd.DataFrame(A, index=[f"g1={int(x)}" for x in grid],
                      columns=[f"g2={int(x)}" for x in grid])
dfB = pd.DataFrame(B, index=[f"g1={int(x)}" for x in grid],
                      columns=[f"g2={int(x)}" for x in grid])

print("\nRow player's payoff matrix A (rows=g1, cols=g2):")
display(dfA)
print("\nColumn player's payoff matrix B (rows=g1, cols=g2):")
display(dfB)

# Also print the 2x2 extreme corner for illustration (g in {0,100})
corner_idx = [0, -1]
print("\nIllustrative corner submatrix (g in {0,100}):")
cornerA = dfA.iloc[corner_idx, corner_idx]
cornerB = dfB.iloc[corner_idx, corner_idx]
display(pd.concat(
    {"A (row player's payoff)": cornerA, "B (col player's payoff)": cornerB},
    axis=1
))

# -----------------------------
# 4) Solve for Nash equilibria using NashPy
# -----------------------------
G = nash.Game(A, B)
print("\nGame summary:", G)

# Support enumeration (works well for 2-player finite games)
equilibria = list(G.support_enumeration())

if equilibria:
    print("\nNash equilibria found (mixed strategies over the discrete grid):")
    for k, (sigma_r, sigma_c) in enumerate(equilibria, 1):
        # Compute expected payoffs at the equilibrium
        u_row = sigma_r @ A @ sigma_c
        u_col = sigma_r @ B @ sigma_c
        print(f"  EQ {k}:")
        print(f"    Row strategy  (over g1 grid {grid.tolist()}): {np.round(sigma_r, 4)}")
        print(f"    Col strategy  (over g2 grid {grid.tolist()}): {np.round(sigma_c, 4)}")
        print(f"    Expected payoffs: (Row={u_row:.4f}, Col={u_col:.4f})")
else:
    print("\nNo equilibrium found by support enumeration on this grid.")

# -----------------------------
# 5) Identify pure-strategy NE explicitly (sanity check)
# -----------------------------
def best_responses_to_g2(g2_idx):
    # Row player best responses given a fixed column action
    col = A[:, g2_idx]
    max_val = np.max(col)
    return np.where(np.isclose(col, max_val))[0]

def best_responses_to_g1(g1_idx):
    # Column player best responses given a fixed row action
    row = B[g1_idx, :]
    max_val = np.max(row)
    return np.where(np.isclose(row, max_val))[0]

pure_NE = []
for i in range(len(grid)):
    br_col = best_responses_to_g1(i)
    for j in range(len(grid)):
        br_row = best_responses_to_g2(j)
        if i in br_row and j in br_col:
            pure_NE.append((i, j))

print("\nPure-strategy NE (indices on the grid):", pure_NE)
if pure_NE:
    for (i, j) in pure_NE:
        print(f"  NE at (g1={grid[i]}, g2={grid[j]}): payoffs (Row={A[i,j]:.2f}, Col={B[i,j]:.2f})")

# -----------------------------
# 6) Social welfare table & dominance illustration
# -----------------------------
W = A + B  # total welfare at each action profile
dfW = pd.DataFrame(W, index=[f"g1={int(x)}" for x in grid],
                      columns=[f"g2={int(x)}" for x in grid])

print("\nTotal welfare W = A + B:")
display(dfW)

max_W = dfW.to_numpy().max()
argmax_idx = np.argwhere(np.isclose(dfW.to_numpy(), max_W))
print(f"Max total welfare value: {max_W:.2f}")
for (i, j) in argmax_idx:
    print(f"  Achieved at (g1={grid[i]}, g2={grid[j]})")

# -----------------------------
# 7) (Optional) Robustness: try a finer grid
# -----------------------------
def run_with_grid(custom_grid):
    print("\n=== Robustness run with custom grid:", custom_grid.tolist(), "===")
    g = custom_grid
    A2 = np.zeros((len(g), len(g)))
    B2 = np.zeros_like(A2)
    for i, g1 in enumerate(g):
        for j, g2 in enumerate(g):
            A2[i, j] = payoff_i(g1, g2)
            B2[i, j] = payoff_i(g2, g1)
    G2 = nash.Game(A2, B2)
    eqs = list(G2.support_enumeration())
    print(f"Found {len(eqs)} equilibrium(a).")
    if eqs:
        for k, (sr, sc) in enumerate(eqs, 1):
            u_r = sr @ A2 @ sc
            u_c = sr @ B2 @ sc
            print(f"  EQ {k}: u_row={u_r:.4f}, u_col={u_c:.4f}")
    # Quick pure-NE scan:
    pure = []
    for i in range(len(g)):
        for j in range(len(g)):
            # row best response to g2=j?
            if np.isclose(A2[:, j], A2[:, j].max()).nonzero()[0].tolist().count(i) > 0 and \
               np.isclose(B2[i, :], B2[i, :].max()).nonzero()[0].tolist().count(j) > 0:
                pure.append((i, j))
    if pure:
        for (i, j) in pure:
            print(f"  Pure NE at (g1={g[i]}, g2={g[j]}) with payoffs ({A2[i,j]:.2f}, {B2[i,j]:.2f})")
    else:
        print("  No pure NE on this grid (only mixed or boundary).")

# Example: finer 11-point grid (every 10 units)
finer_grid = np.linspace(0, 100, 11)
# Uncomment to run robustness check:
# run_with_grid(finer_grid)


Parameters: E=100, m=1.5, n=2, MPCR=0.75
Action grid: [0.0, 25.0, 50.0, 75.0, 100.0]

Row player's payoff matrix A (rows=g1, cols=g2):


,g2=0,g2=25,g2=50,g2=75,g2=100
g1=0,100.00,118.75,137.50,156.25,175.00
g1=25,93.75,112.50,131.25,150.00,168.75
g1=50,87.50,106.25,125.00,143.75,162.50
g1=75,81.25,100.00,118.75,137.50,156.25
g1=100,75.00,93.75,112.50,131.25,150.00



Column player's payoff matrix B (rows=g1, cols=g2):


,g2=0,g2=25,g2=50,g2=75,g2=100
g1=0,100.00,93.75,87.50,81.25,75.00
g1=25,118.75,112.50,106.25,100.00,93.75
g1=50,137.50,131.25,125.00,118.75,112.50
g1=75,156.25,150.00,143.75,137.50,131.25
g1=100,175.00,168.75,162.50,156.25,150.00



Illustrative corner submatrix (g in {0,100}):


A (row player's payoff)        B (col player's payoff)       
                          g2=0 g2=100                    g2=0 g2=100
g1=0                     100.0  175.0                   100.0   75.0
g1=100                    75.0  150.0                   175.0  150.0


Game summary: Bi matrix game with payoff matrices:

Row player:
[[100.   118.75 137.5  156.25 175.  ]
 [ 93.75 112.5  131.25 150.   168.75]
 [ 87.5  106.25 125.   143.75 162.5 ]
 [ 81.25 100.   118.75 137.5  156.25]
 [ 75.    93.75 112.5  131.25 150.  ]]

Column player:
[[100.    93.75  87.5   81.25  75.  ]
 [118.75 112.5  106.25 100.    93.75]
 [137.5  131.25 125.   118.75 112.5 ]
 [156.25 150.   143.75 137.5  131.25]
 [175.   168.75 162.5  156.25 150.  ]]

Nash equilibria found (mixed strategies over the discrete grid):
  EQ 1:
    Row strategy  (over g1 grid [0.0, 25.0, 50.0, 75.0, 100.0]): [ 1.  0.  0. -0.  0.]
    Col strategy  (over g2 grid [0.0, 25.0, 50.0, 75.0, 100.0]): [ 1.  0.  0. -0.  0.]
    Expected payoffs: (Row=100.0000, Col=100.0000)

Pure-strategy NE (indices on the grid): [(0, 0)]
  NE at (g1=0.0, g2=0.0): payoffs (Row=100.00, Col=100.00)

Total welfare W = A + B:


,g2=0,g2=25,g2=50,g2=75,g2=100
g1=0,200.0,212.5,225.0,237.5,250.0
g1=25,212.5,225.0,237.5,250.0,262.5
g1=50,225.0,237.5,250.0,262.5,275.0
g1=75,237.5,250.0,262.5,275.0,287.5
g1=100,250.0,262.5,275.0,287.5,300.0


Max total welfare value: 300.00
  Achieved at (g1=100.0, g2=100.0)
